# Rain Analysis
### Purpose
This notebook will look at volunteer trends for reporting rain, adressing the Github issue #54

### Author: 
Hamza El-Saawy
### Date: 
2020-06-14
### Update Date: 
2020-06-14

### Inputs 
This is where you list any files this notebook requires. 
cbc_effort_weather_1900-2018.txt
1.1-circles_to_many_stations_usa_weather_data_20200424213015

### Output Files
A reduced CBC dataset consisiting of only rain (precipitaion) data and an analysis of that data compared to the NOAA GHCN data

## Steps or Proceedures in the notebook 
 - Clean the CBC data
 - Compare to NOAA data
 - Make some plots

## Where the Data will Be Saved 
The project Google Drive, at: https://drive.google.com/drive/folders/1Nlj9Nq-_dPFTDbrSDf94XMritWYG6E2I

In [3]:
# Imports 
import pandas as pd

## This is a Section Header: How to Get Data For the Project
Data for this project is saved in Google Drive. You can start using this data by following the steps below
1. Clone the github repo (which I'm assuming you've done since your reading this)
2. Go to this folder and download it: https://drive.google.com/drive/folders/1Nlj9Nq-_dPFTDbrSDf94XMritWYG6E2I
3. Paste it into the data folder in your local repository. The path should look like this: audubon-cbc/data/Cloud_Data/
4. Voila! You have data. Lets test it.

In [9]:
# ALL File Paths should be declared at the TOP of the notebook
PATH_TO_RAW_CBC_DATA = "../data/Cloud_Data/cbc_effort_weather_1900-2018.txt"

In [10]:
raw_data = pd.read_csv(PATH_TO_RAW_CBC_DATA, encoding = "ISO-8859-1", sep="\t")

In [11]:
raw_data.head()

,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,...,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,am_rain,pm_rain,am_snow,pm_snow
0,Scotch Lake,CA-NB,45.950000,-66.983300,1901,12/25/00,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toronto,CA-ON,43.700000,-79.416700,1901,12/25/00,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pacific Grove,US-CA,36.616700,-121.916700,1901,12/25/00,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pueblo,US-CO,38.175251,-104.519575,1901,12/25/00,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bristol,US-CT,41.671800,-72.949500,1901,12/25/00,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Saving Data From a Notebook
If a notebook is used to clean or add to a dataset, the resulting dataset should be named after the notebook that produced it. For example the data produced by this notebook would be called 0.0-rec-example-notebook-with-header.csv 

## Getting your data into Google Drive. 
If you have data you think is ready to be added to the offical google drive data folder, your DA will ask you for the following infomation.

- Number of Rows in the Data
- Discription and name of any new feilds 